<a href="https://colab.research.google.com/github/alexworeku/Privacy/blob/master/Custom_Preprocessor_GLUE_tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!rm -rf /root/.cache/huggingface/datasets

In [3]:
!pip install --upgrade --force-reinstall datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.8/514.8 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [32]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from functools import partial

In [3]:
checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
dataset = load_dataset("glue","sst2")

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [21]:
TASK_CONFIG = {
    "cola": {
        "text_fields": ("sentence",),
        "use_token_type_ids": False,
        "description": "CoLA (Corpus of Linguistic Acceptability): Binary classification of grammatical acceptability."
    },
    "sst2": {
        "text_fields": ("sentence",),
        "use_token_type_ids": False,
        "description": "SST-2 (Stanford Sentiment Treebank): Binary classification for sentiment analysis (positive or negative)."
    },
    "mrpc": {
        "text_fields": ("sentence1", "sentence2"),
        "use_token_type_ids": True,
        "description": "MRPC (Microsoft Research Paraphrase Corpus): Binary classification to determine if two sentences are semantically equivalent."
    },
    "qqp": {
        "text_fields": ("question1", "question2"),
        "use_token_type_ids": True,
        "description": "QQP (Quora Question Pairs): Binary classification to identify duplicate questions."
    },
    "sts-b": {
        "text_fields": ("sentence1", "sentence2"),
        "use_token_type_ids": True,
        "description": "STS-B (Semantic Textual Similarity Benchmark): Regression task predicting semantic similarity score (0 to 5)."
    },
    "mnli": {
        "text_fields": ("premise", "hypothesis"),
        "use_token_type_ids": True,
        "description": "MNLI (Multi-Genre Natural Language Inference): Multi-class classification (entailment, contradiction, neutral)."
    },
    "mnli-mm": {
        "text_fields": ("premise", "hypothesis"),
        "use_token_type_ids": True,
        "description": "MNLI-mismatched: Same as MNLI but evaluated on a mismatched genre set."
    },
    "qnli": {
        "text_fields": ("question", "sentence"),
        "use_token_type_ids": True,
        "description": "QNLI (Question Natural Language Inference): Binary classification to determine if a sentence answers a question."
    },
    "rte": {
        "text_fields": ("sentence1", "sentence2"),
        "use_token_type_ids": True,
        "description": "RTE (Recognizing Textual Entailment): Binary classification (entailment vs. not entailment)."
    }
}

In [30]:
sst2_config = TASK_CONFIG['sst2']
sst2_config['text_fields']

('sentence',)

In [37]:
def tokenize_samples(sample,text_fields, padding,truncation):
  if len(text_fields) == 1:
    return tokenizer(sample[text_fields[0]], truncation = truncation,padding=padding)

  elif len(text_fields) == 2:
    return tokenizer(sample[text_fields[0]], sample[text_fields[1]], truncation = truncation, padding = padding)


def preprocess_glue_dataset(raw_dataset:Dataset,task_config, padding=False, truncation = True):
  tokenized_dataset = raw_dataset.map(partial(tokenize_samples, text_fields = task_config['text_fields'],
                                              padding = padding, truncation = truncation), batched=True)

  return tokenized_dataset

In [35]:
# Testing preprocessor on sst2 dataset
tokenized_inputs = preprocess_glue_dataset(dataset,TASK_CONFIG['sst2'])
tokenized_inputs

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [39]:
# Testing preprocessor on cola dataset:

cola_dataset = load_dataset('glue','cola')

train-00000-of-00001.parquet:   0%|          | 0.00/251k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/37.6k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/37.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

In [48]:
cola_tokenized_inputs = preprocess_glue_dataset(cola_dataset,TASK_CONFIG['cola'],)

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

In [50]:
cola_tokenized_inputs['train'][0]

{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.",
 'label': 1,
 'idx': 0,
 'input_ids': [101,
  2256,
  2814,
  2180,
  1005,
  1056,
  4965,
  2023,
  4106,
  1010,
  2292,
  2894,
  1996,
  2279,
  2028,
  2057,
  16599,
  1012,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}